# Программирование на Python 
*Алла Тамбовцева*

## Извлечение информации из кода JavaScript внутри HTML

Импортируем библиотеки и функцию `BeautifulSoup` (понадобятся для разных примеров ниже):

* `requests` для отправки запроса и получения кода HTML веб-страницы;
* `bs4` для поиска тэгов в коде HTML;
* `pandas` для обработки полученной информации и приведения ее к табличному виду.

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

У нас было [домашнее задание](https://nbviewer.org/url/python.math-hse.info/static/assignments_release/pyperm/pyperm-hw06/pyperm-hw06.ipynb) на парсинг страницы фильма «Не покидай...» с сайта www.kino-teatr.ru. Сайт некоммерческий, довольно дружелюбный, позволяет свободно выгружать информацию. Но у него есть одна особенность: число лайков и дизлайков, поставленных актерам пользователями, загружается на страницу динамически, то есть автоматически «подтягивается» с сервера при загрузке страницы в определенный момент времени. На практике это выливается в то, что найти нужную информацию по тэгам просто невозможно, ее нет в основном коде HTML. Как быть? Понять, как выглядит запрос данных, который отправляется на сервер, и выяснить, где хранятся нужные нам данные. Мы рассмотрим несложный случай, когда сайт забирает информацию из строки JSON, которая находится на странице, но внутри кода, написанного на JavaScript. Такое можно встретить на страницах с результатами каких-нибудь игр или на сайтах, посвященных динамике цен или курсу валют (другой вопрос, что не всегда JSON прямо так явно находится в том же файле, где и код HTML).

Так как ранее мы обсуждали довольно базовый парсинг, в домашнем задании не требовалось собирать число лайков и дизлайков, эти значения были даны в виде готового списка словарей. Обновим задачу – теперь нам нужно собрать имена актеров и их id, а затем «подтянуть» к этой информации число голосов за и против. 

Начало работы стандартное – загружаем код HTML страницы по ссылке и преобразуем его в объект BeautifulSoup:

In [3]:
page = requests.get("https://www.kino-teatr.ru/kino/movie/sov/4319/titr/")
soup = BeautifulSoup(page.text)

Ищем имена актеров – находим блоки с тэгами `<div>` с классом `film_name` и вытаскиваем из них «чистый» текст:

In [4]:
names_raw = soup.find_all("div", {"class" : "film_name"}) 
names = [name.text for name in names_raw]

print(names[0:10]) # первые 10 для примера

['Лидия Федосеева-Шукшина', 'Вячеслав Невинный', 'Игорь Красавин', 'Варвара Владимирова', 'Светлана Селезнёва', 'Регина Разума', 'Альберт Филозов', 'Артём Тынкасов', 'Елена Антонова', 'Владимир Ставицкий']


Теперь ищем id, они нам понадобятся для совмещения с информацией по числу голосов за и против:

In [5]:
divs = soup.find_all("div", {"class" : "actor_film_descript"}) 
print(divs[0:2])

[<div class="actor_film_descript" id="role_16801">
<div class="film_name"><a href="/kino/acter/w/sov/4484/bio/" itemprop="url" title="Лидия Федосеева-Шукшина"><strong itemprop="name">Лидия Федосеева-Шукшина</strong></a></div>
<div class="film_role">Королева Флора — <span class="film_main_role">главная роль</span></div>
<div class="film_role_descript">жена короля Теодора</div>
<div class="film_rating"><span id="role_rating_16801"></span></div>
</div>, <div class="actor_film_descript" id="role_16800">
<div class="film_name"><a href="/kino/acter/m/sov/3035/bio/" itemprop="url" title="Вячеслав Невинный"><strong itemprop="name">Вячеслав Невинный</strong></a></div>
<div class="film_role">Король Теодор — <span class="film_main_role">главная роль</span></div>
<div class="film_rating"><span id="role_rating_16800"></span></div>
</div>]


Как можно заметить, числовых id здесь нет, но это легко исправить – забрать значения атрибута `id` через метод `.get()` (вспоминаем о сходстве объектов BeautifulSoup и словарей), разбить их по символу `_` и забрать часть после `_` с индексом 1:

In [6]:
ids = [i.get("id").split("_")[1] for i in divs]
print(ids[0:10])

['16801', '16800', '16803', '16802', '89473', '124124', '16804', '132138', '56008', '132139']


Теперь воспользуемся тем, что функция `DataFrame()` из библиотеки pandas умеет превращать в датафрейм не только списки списков или словари, но и списки кортежей. Объединим элементы в список попарно через функцию `zip()` и сконвертируем перечень пар-кортежей в датафрейм:

In [7]:
# напоминание: как выглядят элементы в zip()

list(zip(ids, names))[0:10]

[('16801', 'Лидия Федосеева-Шукшина'),
 ('16800', 'Вячеслав Невинный'),
 ('16803', 'Игорь Красавин'),
 ('16802', 'Варвара Владимирова'),
 ('89473', 'Светлана Селезнёва'),
 ('124124', 'Регина Разума'),
 ('16804', 'Альберт Филозов'),
 ('132138', 'Артём Тынкасов'),
 ('56008', 'Елена Антонова'),
 ('132139', 'Владимир Ставицкий')]

In [8]:
main = pd.DataFrame(zip(ids, names))
main.columns = ["id", "name"] 
main

,id,name
0,16801,Лидия Федосеева-Шукшина
1,16800,Вячеслав Невинный
2,16803,Игорь Красавин
3,16802,Варвара Владимирова
4,89473,Светлана Селезнёва
5,124124,Регина Разума
6,16804,Альберт Филозов
7,132138,Артём Тынкасов
8,56008,Елена Антонова
9,132139,Владимир Ставицкий


Теперь переходим к более сложной части – поиску голосов за и против. Просто найти на странице кнопки красного и зеленого цвета и забрать с них текст не получится:

![](НП.jpeg)

Поэтому для этого на нужно найти код JavaScript, где есть записи с числами `plus` и `minus` с привязкой к id актеров. Код JavaScript, если он не вынесен в отдельный файл, заключается в тэги `<script>`. Если мы внимательно изучим исходный код страницы, мы заметим, что нужный нам блок кода имеет атрибут `type` со значением `text/javascript`, и среди таких блоков он находится на 7-ом месте:

In [10]:
# выполняем поиск и извлекаем текст – код JavaScript в виде строки

text = soup.find_all("script", {"type" : "text/javascript"})[6].text 
#text

Блок с кодом довольно большой, в нем содержатся разные функции для отрисовки и обновления информации на кнопках (на них отображаются голоса за и против, на них же можно кликать после авторизации, чтобы записать свой голос). Нам же понадобится часть кода после функции `print_role_rating_buttons ()`, так как именно под ней располагается необходимая информация. Воспользуемся методом `.split()` и разобьем строку на части:

In [11]:
for_votes = text.split("function print_all_role_rating_buttons ()")[1]
print(for_votes)


{
    print_role_rating_buttons({ id:"160325", plus:"52", minus:"2", voted:"" });
print_role_rating_buttons({ id:"1928641", plus:"25", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1966145", plus:"21", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973000", plus:"21", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973001", plus:"21", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973002", plus:"23", minus:"0", voted:"" });
print_role_rating_buttons({ id:"1973003", plus:"23", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973005", plus:"23", minus:"2", voted:"" });
print_role_rating_buttons({ id:"1973007", plus:"26", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973008", plus:"20", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973010", plus:"31", minus:"5", voted:"" });
print_role_rating_buttons({ id:"1973011", plus:"42", minus:"2", voted:"" });
print_role_rating_buttons({ id:"2088754", plus:"27", minus:"2", voted:

Хотя с синтаксисом JavaScript знакомы немногие, не очень сложно догадаться, что означают записи такого вида:

    print_role_rating_buttons({ id:"16800", plus:"140", minus:"4", voted:"" });
    
Такая строка кода активирует функцию `print_role_rating_buttons()` – применяет ее к такому набору данных и наносит на кнопки, соответствующие актеру с id 16800 значения 140 (зеленая кнопка) и 4 (красная кнопка):

![](ВН.jpeg)

*Примечание 1*. Синхронизация не происходит мгновенно, актуальное значение 140 подставляется на кнопку не сразу, там вполне может стоять значение 139, а мы, благодаря значениям из JavaScript, обладаем более актуальной информацией. Это стоит иметь в виду при выгрузке информации, когда мы сверяем полученные результаты и то, что видим на экране. Ну и, конечно, если времени с момента скриншота прошло больше, значения успели измениться :)

*Примечание 2*. Значение `voted` в коде выше является непустым, в случае, если пользователь зарегистрирован и оценивал игру актера (по крайней мере, какое-то время после голосования значение в `voted` держится, потом на самой странице отметка о голосовании остается, а в JSON стирается). Сравним скрин и фрагменты кода для такого случая:

![](ВС_ЕА.jpeg)
    
Код:

    print_role_rating_buttons({ id:"56008", plus:"90", minus:"10", voted:"plus" });
    print_role_rating_buttons({ id:"132139", plus:"106", minus:"4", voted:"plus" });


Все фрагменты кода выше – это обычные строки, объекты типа *string*. Поэтому с помощью регулярных выражений мы сможем найти в них записи в фигурных скобках, затем сделать из них словари, а из списка словарей с единообразными ключами собрать датафрейм!

Найдем все подстроки, соответствующие шаблону `\{.+\}`, то есть просто все наборы символов, заключенных в фигурные скобки (скобки экранируем, чтобы не перепутать со специальными скобками в регулярных выражениях):

In [26]:
import re

In [13]:
votes_str = re.findall("\{.+\}", for_votes)
votes_str

['{ id:"160325", plus:"52", minus:"2", voted:"" }',
 '{ id:"1928641", plus:"25", minus:"1", voted:"" }',
 '{ id:"1966145", plus:"21", minus:"1", voted:"" }',
 '{ id:"1973000", plus:"21", minus:"1", voted:"" }',
 '{ id:"1973001", plus:"21", minus:"1", voted:"" }',
 '{ id:"1973002", plus:"23", minus:"0", voted:"" }',
 '{ id:"1973003", plus:"23", minus:"1", voted:"" }',
 '{ id:"1973005", plus:"23", minus:"2", voted:"" }',
 '{ id:"1973007", plus:"26", minus:"1", voted:"" }',
 '{ id:"1973008", plus:"20", minus:"1", voted:"" }',
 '{ id:"1973010", plus:"31", minus:"5", voted:"" }',
 '{ id:"1973011", plus:"42", minus:"2", voted:"" }',
 '{ id:"2088754", plus:"27", minus:"2", voted:"" }',
 '{ id:"16800", plus:"144", minus:"4", voted:"" }',
 '{ id:"16801", plus:"100", minus:"24", voted:"" }',
 '{ id:"16803", plus:"121", minus:"12", voted:"" }',
 '{ id:"16802", plus:"135", minus:"7", voted:"" }',
 '{ id:"16804", plus:"134", minus:"4", voted:"" }',
 '{ id:"56008", plus:"94", minus:"10", voted:"" }'

Выберем один элемент списка и изучим его:

In [14]:
v = votes_str[0]
v

'{ id:"160325", plus:"52", minus:"2", voted:"" }'

На первый взгляд, этот элемент представляет собой полноценный словарь, просто заключенный в кавычки. 
Однако есть проблема: чтобы переделать строку в словарь, нужна строка, соответствующая формату JSON, а здесь не хватает кавычек вокруг ключей. Как эту проблему решить? Найти по какому-то паттерну эти ключи и доклеить вокруг них кавычки с помощью функции `sub()` из модуля `re` для «умной» замены (не обычная замена, так как мы не конкретный набор символов заменяем на другой, а ищем совпадения по некоторому общему шаблону и их изменяем). 

Поиск будет простой – в строке с «неправильным» словарем содержатся либо цифры, либо буквы, либо пробелы с запятыми и знаками препинания, а нам нужны последовательности из одной и более букв:

In [15]:
re.findall("[a-z]+", v)

['id', 'plus', 'minus', 'voted']

Напишем функцию `add_quotes()`, которая принимает на вход строку `x`, доклеивает к ней кавычки и убирает пробелы. Нам нужно преобразовать строку `x` в группу – объект из регулярных выражений – через `+` доклеить кавычки и на всякий случай убрать лишние пробелы:

In [16]:
# x.group(): функция sub() из re для умной замены
# работает не со строками, а с объектом специального типа – группа символов

def add_quotes(x):
    g = '"' + x.group() + '"'
    return g.strip()

Применяем функцию `sub` к одной строке `v` и проверяем:

In [17]:
# аргументы:
# шаблон, по которому ищем, что заменять
# функция, которая выполняет преобразование – доклеивает к результатм поиска кавычки
# строка, по который выполняем поиск

re.sub("[a-z]+", add_quotes, v)

'{ "id":"160325", "plus":"52", "minus":"2", "voted":"" }'

Все работает, применяем ко всем строкам и получаем список валидных json-строк:

In [18]:
votes_new = [re.sub("[a-z]+", add_quotes, v) for v in votes_str]
votes_new

['{ "id":"160325", "plus":"52", "minus":"2", "voted":"" }',
 '{ "id":"1928641", "plus":"25", "minus":"1", "voted":"" }',
 '{ "id":"1966145", "plus":"21", "minus":"1", "voted":"" }',
 '{ "id":"1973000", "plus":"21", "minus":"1", "voted":"" }',
 '{ "id":"1973001", "plus":"21", "minus":"1", "voted":"" }',
 '{ "id":"1973002", "plus":"23", "minus":"0", "voted":"" }',
 '{ "id":"1973003", "plus":"23", "minus":"1", "voted":"" }',
 '{ "id":"1973005", "plus":"23", "minus":"2", "voted":"" }',
 '{ "id":"1973007", "plus":"26", "minus":"1", "voted":"" }',
 '{ "id":"1973008", "plus":"20", "minus":"1", "voted":"" }',
 '{ "id":"1973010", "plus":"31", "minus":"5", "voted":"" }',
 '{ "id":"1973011", "plus":"42", "minus":"2", "voted":"" }',
 '{ "id":"2088754", "plus":"27", "minus":"2", "voted":"" }',
 '{ "id":"16800", "plus":"144", "minus":"4", "voted":"" }',
 '{ "id":"16801", "plus":"100", "minus":"24", "voted":"" }',
 '{ "id":"16803", "plus":"121", "minus":"12", "voted":"" }',
 '{ "id":"16802", "plus":"

Теперь нам осталось считать эти валидные JSON-строки с помощью Python – то есть превратить их в обычные питоновские словари.  Импортируем модуль `json`:

In [19]:
import json

Проверяем работу функции `loads()` на примере первой строки из полученного списка `votes_new`:

In [20]:
json.loads(votes_new[0])

{'id': '160325', 'plus': '52', 'minus': '2', 'voted': ''}

Напоминание: в модуле `json` есть две похожих функции:

* `load()` загружает данные из файла с расширением `.json`;
* `loads()` – загружает данные из JSON-строки.

Теперь применим эту функцию ко всем строкам в `votes_new` и сразу преобразуем получившийся список словарей в датафрейм:

In [21]:
ratings = pd.DataFrame([json.loads(v) for v in votes_new])
ratings

,id,plus,minus,voted
0,160325,52,2,
1,1928641,25,1,
2,1966145,21,1,
3,1973000,21,1,
4,1973001,21,1,
5,1973002,23,0,
6,1973003,23,1,
7,1973005,23,2,
8,1973007,26,1,
9,1973008,20,1,


Ура! Вся относительно тяжелая работа проделана, осталось вспомнить, что у нас есть датафрейм `main`, в который мы хотели добавить информацию о числе лайков и дизлайков:

In [22]:
main.head()

,id,name
0,16801,Лидия Федосеева-Шукшина
1,16800,Вячеслав Невинный
2,16803,Игорь Красавин
3,16802,Варвара Владимирова
4,89473,Светлана Селезнёва


Объединим два датафрейма, `main` и `ratings` по столбцу с названием `id` с помощью метода `.merge()`. Нам понадобится тип объединения `left`, так как мы не хотим терять информацию по тем актерам, по которым голосования на странице не было, а хотим просто к первому (левому) датафрейму подтянуть информацию из второго (правого). Если такой информации нет, строка из `main` не удалится, просто на соответствующих местах в объединенном датафрейме будут пропуски:

In [23]:
final = main.merge(ratings, on = "id", how = "left")
final 

,id,name,plus,minus,voted
0,16801,Лидия Федосеева-Шукшина,100,24,
1,16800,Вячеслав Невинный,144,4,
2,16803,Игорь Красавин,121,12,
3,16802,Варвара Владимирова,135,7,
4,89473,Светлана Селезнёва,94,20,
5,124124,Регина Разума,122,7,
6,16804,Альберт Филозов,134,4,
7,132138,Артём Тынкасов,125,2,
8,56008,Елена Антонова,94,10,
9,132139,Владимир Ставицкий,110,5,


Отсортируем строки по числу положительных оценок:

In [24]:
final.sort_values("plus")

,id,name,plus,minus,voted
16,132143,Вика Яблонская,0,0,
0,16801,Лидия Федосеева-Шукшина,100,24,
9,132139,Владимир Ставицкий,110,5,
10,72744,Анатолий Рудаков,118,3,
2,16803,Игорь Красавин,121,12,
5,124124,Регина Разума,122,7,
7,132138,Артём Тынкасов,125,2,
6,16804,Альберт Филозов,134,4,
3,16802,Варвара Владимирова,135,7,
1,16800,Вячеслав Невинный,144,4,


Результат сортировки получился странным: логика сортировки не похожа ни на убывание, ни на возрастание.

Почему так вышло? Проблема в том, что тип столбцов `plus` и `minus` остался строковым, а строки сортируются посимвольно: сравниваются первые символы, потом вторые, потом третьи... Сортировка по умолчанию идет по возрастанию, поэтому сначала идут строки, начинающиеся с 0 и 1, затем – с 2, и так далее, а внутри каждой группы сортировка проходит по второму символу, и если нужно, по третьему.

Преобразуем тип столбца в числовой и снова отсортируем, теперь уже точно правильно и по убыванию (тип столбца `float`, так как сделать столбец с пропусками `NaN` целочисленным pandas не позволит):

In [25]:
final["plus"] = final["plus"].astype(float) 
final["minus"] = final["minus"].astype(float) 

final.sort_values("plus", ascending = False)

,id,name,plus,minus,voted
1,16800,Вячеслав Невинный,144.0,4.0,
3,16802,Варвара Владимирова,135.0,7.0,
6,16804,Альберт Филозов,134.0,4.0,
7,132138,Артём Тынкасов,125.0,2.0,
5,124124,Регина Разума,122.0,7.0,
2,16803,Игорь Красавин,121.0,12.0,
10,72744,Анатолий Рудаков,118.0,3.0,
9,132139,Владимир Ставицкий,110.0,5.0,
0,16801,Лидия Федосеева-Шукшина,100.0,24.0,
12,62460,Александр Денисов,97.0,4.0,


Отлично, задача решена!